<a href="https://colab.research.google.com/github/zzqnot996/DeepLearning/blob/main/VarifocalLoss_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 【目标检测实验系列】YOLOv5改进实验：结合VariFocal Loss损失函数，减少小目标漏检问题，高效提升模型检测的召回率（超详细改进代码流程）

https://blog.csdn.net/qq_32575047/article/details/130454459

2. focal loss 通俗讲解

https://zhuanlan.zhihu.com/p/266023273

3. ValFocalLoss VarifocalNet 或VFNet

https://blog.csdn.net/calvinpaean/article/details/115497134  4.2节




VarifocalNet: An IoU-aware Dense Object Detector（CVPR 2021）

paper：VarifocalNet: An IoU-aware Dense Object Detector

official implementation：https://github.com/hyz-xmaster/VarifocalNet

third-party implementation：mmdetection/vfnet_head.py at main · open-mmlab/mmdetection  


In [ ]:
import torch
from torch import nn

class VFLoss(nn.Module):
    def __init__(self, loss_fcn, gamma=1.5, alpha=0.25):
        super(VFLoss, self).__init__()
        # 传递 nn.BCEWithLogitsLoss() 损失函数  must be nn.BCEWithLogitsLoss()
        self.loss_fcn = loss_fcn  #
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = loss_fcn.reduction
        self.loss_fcn.reduction = 'none'  # required to apply VFL to each element

    def forward(self, pred, true):

        loss = self.loss_fcn(pred, true)

        pred_prob = torch.sigmoid(pred)  # prob from logits

        focal_weight = true * (true > 0.0).float() + self.alpha * (pred_prob - true).abs().pow(self.gamma) * (true <= 0.0).float()
        loss *= focal_weight

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss
